In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import evallm
from automata.fa.dfa import DFA
import tqdm.auto as tqdm

In [3]:
import itertools
from permacache import stable_hash

In [4]:
from evallm.experiments.transducer_plotting import produce_table

from evallm.experiments.transducer_summary import compute_results, prompt_by_key

In [5]:
accuracies = compute_results()

In [6]:
prompts = list(prompt_by_key)

In [7]:
def display_prompt(p):
    return rf"\textsc{{{p}}}$_T$"

In [8]:
produce_table(
    {
        m: {display_prompt(p): accuracies[m][p] for p in accuracies[m]}
        for m in accuracies
    },
    [display_prompt(p) for p in prompt_by_key],
)

\begin{tabular}{|r|c|c|c|c|}
\hline
Model & \textsc{Basic}$_T$ & \textsc{More-Expl}$_T$ & \textsc{COT}$_T$ & \textsc{Red-Green}$_T$\\
\hline
\cellcolor{lightgray}\textsc{BruteForce}$_T$ &\cellcolor{lightgray}96.4 (96.2--96.7)&--&--&--\\
\hline
\bf 6-\textsc{Gram}$_T$ &\bf 93.5 (93.1--93.9)&--&--&--\\
\hline
5-\textsc{Gram}$_T$ &93.4 (93.0--93.7)&--&--&--\\
\hline
4-\textsc{Gram}$_T$ &91.1 (90.6--91.6)&--&--&--\\
\hline
mistral-nemo-minitron-8B &88.6 (88.0--89.1)&--&--&--\\
\hline
qwen-2.5-coder-instruct-7B &88.3 (87.8--88.8)&--&--&--\\
\hline
qwen-2.5-coder-7B &88.2 (87.6--88.7)&--&--&--\\
\hline
mistral-nemo-instruct-12B &88.0 (87.5--88.5)&--&--&--\\
\hline
qwen-2.5-coder-instruct-32B &87.9 (87.4--88.4)&--&--&--\\
\hline
mistral-nemo-base-12B &87.9 (87.4--88.4)&--&--&--\\
\hline
gpt-3.5-instruct &87.8 (85.9--89.6)&--&--&--\\
\hline
llama3-70B &87.7 (87.2--88.3)&--&--&--\\
\hline
starcoder2-15b &87.7 (85.8--89.5)&--&--&--\\
\hline
llama3-8B &87.5 (86.9--88.0)&--&--&--\\
\hline
claude-3